# Ensemble unsupervised learning

In [1]:
import pandas as pd
import numpy as np
import os
import json
import torch
import torch.nn as nn
from utils import Packet, Flow
from config import whisper_config
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

MAX_LEN = whisper_config["n_fft"] * 2

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Useful functions

In [2]:
def get_flows(df: pd.DataFrame, key_type: str = "default") -> dict:
    mp = dict()
    for idx in range(len(df)): # simulate the process of packet processing
        row = df.iloc[idx]
        pkt = Packet(
            src_ip=row["src_ip"],
            dst_ip=row["dst_ip"],
            src_port=row["src_port"],
            dst_port=row["dst_port"],
            protocol=row["protocol"],
            proto_code=row["proto_code"],
            pkt_length=row["pkt_length"],
            timestamp=row["timestamp"],
            ttl=row["ttl"],
            tcp_window=row["tcp_window"],
            tcp_dataoffset=row["tcp_dataoffset"],
            udp_length=row["udp_length"],
            label=row["label"],
        )
        key = pkt.key(type=key_type)
        if key not in mp:
            mp[key] = Flow()
        mp[key].add_packet(pkt)
    return mp

In [14]:
def transform(mp: dict, feature_type: str = "whisper", 
              data_type: str = "train", test_data_aug: bool = True):
    packet_data, flow_data = [], []
    packet_labels, flow_labels = [], []
    for key, flow in mp.items():
        vec = flow.vector()
        if feature_type == "whisper":
            if len(vec) <= (whisper_config["n_fft"] // 2):
                # packet level features
                # vec = flow.packet_vector(agg_type="mean") + flow.packet_vector(agg_type="std") \
                #     + flow.packet_vector(agg_type="max") + flow.packet_vector(agg_type="min")
                # packet_data.append(vec)
                # packet_labels.append(flow.label)

                # implement fft on short flows
                ten = torch.tensor(vec)
                ten_fft = torch.fft.fft(ten, n=(whisper_config["n_fft"] // 2)+1)
                ten_power = torch.pow(ten_fft.real, 2) + torch.pow(ten_fft.imag, 2)
                ten_res = (ten_power.squeeze()+1).log2()
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                if data_type == "test" and test_data_aug:
                    # data shape for test data augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    packet_data.append([ten_res.tolist()])
                else:
                    # data shape for no data augmentation: (n_flow, floor(n_fft/2)+1)
                    packet_data.append(ten_res.tolist())
                packet_labels.append(flow.label)
                
            else:
                # flow level featrues
                ten = torch.tensor(vec)
                # stft requirement: input_size > (n_fft // 2)
                # default return shape: (floor(n_fft/2)+1, n_frame, 2)
                ten_fft = torch.stft(ten, whisper_config["n_fft"])
                ten_power = torch.pow(ten_fft[:,:,0], 2) + torch.pow(ten_fft[:,:,1], 2)
                ten_res = ((ten_power.squeeze()+1).log2()).permute(1,0)
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                # ten_res shape: (n_frame, floor(n_fft/2)+1)
                if data_type == "train":
                    if (ten_res.size(0) > whisper_config["mean_win_train"]):
                        for _ in range(whisper_config["num_train_sample"]):
                            start_idx = torch.randint(0, ten_res.size(0)
                                        - whisper_config["mean_win_train"], (1,)).item()
                            ten_tmp = ten_res[start_idx:start_idx+whisper_config["mean_win_train"],:].mean(dim=0)
                            flow_data.append(ten_tmp.tolist())
                            flow_labels.append(flow.label)
                    else:
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        flow_labels.append(flow.label)
                else: # for test
                    if test_data_aug:
                        tmp_data = []
                        if (ten_res.size(0) > whisper_config["mean_win_test"]):
                            # data augmentation for kmeans on flows with length > mean_win_test
                            for idx in range(0, ten_res.size(0) - whisper_config["mean_win_test"], 
                                            whisper_config["mean_win_test"]):
                                ten_tmp = ten_res[idx:idx+whisper_config["mean_win_test"],:].mean(dim=0)
                                tmp_data.append(ten_tmp.tolist())
                        else:
                            # no data augmentation for kmeans on flows with length < mean_win_test
                            tmp_data.append(ten_res.mean(dim=0).tolist())
                        flow_data.append(tmp_data)
                        # data shape for augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    else: # for other detection methods
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        # data shape for no augmentation: (n_flow, floor(n_fft/2)+1)
                    flow_labels.append(flow.label)
        elif feature_type == "encoding":
            # directly use the whisper encoding vector
            if len(vec) >= MAX_LEN:
                new_vec = vec[:MAX_LEN]
            else:
                new_vec = vec + [0] * (MAX_LEN - len(vec))
            if len(vec) <= (whisper_config["n_fft"] // 2):
                packet_data.append(new_vec)
                packet_labels.append(flow.label)
            else:
                flow_data.append(new_vec)
                flow_labels.append(flow.label)
        else: # for other feature types
            pass
    return packet_data, packet_labels, flow_data, flow_labels

In [4]:
benign_filenames = [os.path.join("train_set", "benign" + str(i) + ".csv") 
                    for i in range(1, 3)]
attack_filenames = [os.path.join("attack_set", x) for x in 
                    os.listdir("attack_set") if x.endswith(".csv")]

In [51]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=None, decoder_sigmoid=False):
        super(AutoEncoder, self).__init__()
        if hidden_dim is None:
            hidden_dim = int(0.75 * input_dim)
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Sigmoid())
        if decoder_sigmoid:
            self.decoder = nn.Sequential(
                nn.Linear(hidden_dim, input_dim),
                nn.Sigmoid())
        else:
            self.decoder = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        super(Dataset, self).__init__()
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [25]:
def train_kmeans(train_data, kmeans_save_path, n_clusters=10):
    train_data = torch.tensor(train_data).float()
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(train_data)
    centroids = torch.tensor(kmeans.cluster_centers_).float()
    train_loss = torch.cdist(train_data, centroids, p=2).min(dim=1).values.mean()
    if not os.path.exists(os.path.dirname(kmeans_save_path)):
        os.makedirs(os.path.dirname(kmeans_save_path))
    with open(kmeans_save_path, "w") as f:
        json.dump({
            "centroids": centroids.tolist(),
            "train_loss": train_loss.item()
        }, f, indent=4)
    

In [17]:
def train_ae(train_data, train_labels, save_dir,
            model, criterion, optimizer, device, 
            batch_size=32, num_epochs=200, 
            decoder_sigmoid=False):
    train_dataset = Dataset(train_data, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    loss_list = []
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for data, labels in train_loader:
            if decoder_sigmoid:
                data = torch.sigmoid(data.to(device).float())
            else:
                data = data.to(device).float()
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, data)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    os.makedirs(save_dir, exist_ok=True)
    model_save_path = os.path.join(save_dir, "model.pt")
    torch.save(model.state_dict(), model_save_path)
    loss_save_path = os.path.join(save_dir, "train_loss.json")
    with open(loss_save_path, "w") as f:
        json.dump(loss_list, f)

In [7]:
def test_kmeans(data, kmeans_load_path, whisper_config, scale=10):
    with open(kmeans_load_path, "r") as f:
        model_param = json.load(f)
    centroids = torch.tensor(model_param["centroids"])
    train_loss = model_param["train_loss"]

    kmeans_preds, kmeans_ratios = [], []
    for val in data:
        val = torch.tensor(val)
        if (val.size(0) > whisper_config["mean_win_test"]):
            max_dist = 0
            for idx in range(0, val.size(0) - whisper_config["mean_win_test"], 
                             whisper_config["mean_win_test"]):
                ten_tmp = val[idx:idx+whisper_config["mean_win_test"],:].mean(dim=0)
                dist = torch.norm(ten_tmp - centroids, dim=1).min()
                max_dist = max(max_dist, dist)
            min_dist = max_dist
        else:
            min_dist = torch.norm(val.mean(dim=0) - centroids, dim=1).min()
        kmeans_preds.append(-1 if min_dist > scale * train_loss else 1)
        kmeans_ratios.append(min_dist/(scale * train_loss))
    return kmeans_preds, kmeans_ratios

In [8]:
def test_ae(test_data, model, device, criterion, 
            threshold, scale=5, test_data_aug=False,
            decoder_sigmoid=False):
    model.eval()
    preds, ratios = [], []
    with torch.no_grad():
        for val in test_data:
            if decoder_sigmoid:
                data = torch.sigmoid(torch.tensor(val).to(device)).float()
            else:
                data = torch.tensor(val).to(device).float()
            outputs = model(data)
            loss = criterion(outputs, data)
            if not test_data_aug:
                ratios.append(loss.item()/(scale * threshold))
                preds.append(-1 if loss.item() > threshold * scale else 1)
            else:
                ratios.append(loss.max().item()/(scale * threshold))
                preds.append(-1 if loss.max().item() > threshold * scale else 1)
    return preds, ratios

In [75]:
def test_ensemble(datac, dataw, labels, kmeans_load_path,
         aec_input_dim, aec_load_path, aew_input_dim, aew_load_path, 
         kmeans_scale=7, aec_scale=10, aew_scale=3,
         test_data_aug=False, majority_vote=False):
    
    kmeans_preds, kmeans_ratios = test_kmeans(dataw, kmeans_load_path, 
                                              whisper_config, scale=kmeans_scale)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.MSELoss()

    model_aec = AutoEncoder(aec_input_dim, decoder_sigmoid=True)
    model_aec.load_state_dict(torch.load(os.path.join(aec_load_path, "model.pt")))
    model_aec.to(device)
    with open(os.path.join(aec_load_path, "train_loss.json"), "r") as f:
        loss_list = json.load(f)
    threshold = torch.tensor(loss_list).mean().item()
    aec_preds, aec_ratios = test_ae(datac, model_aec, device, criterion, threshold, 
                                    scale=aec_scale, test_data_aug=False, 
                                    decoder_sigmoid=True) 
    
    model_aew = AutoEncoder(aew_input_dim)
    model_aew.load_state_dict(torch.load(os.path.join(aew_load_path, "model.pt")))
    model_aew.to(device)
    with open(os.path.join(aew_load_path, "train_loss.json"), "r") as f:
        loss_list = json.load(f)
    threshold = torch.tensor(loss_list).mean().item()
    aew_preds, aew_ratios = test_ae(dataw, model_aew, device, criterion, threshold, 
                                    scale=aew_scale, test_data_aug=test_data_aug, 
                                    decoder_sigmoid=False)

    # preds = np.sign(np.array(kmeans_preds) + np.array(aec_preds) + np.array(aew_preds))
    preds = []
    for idx in range(len(kmeans_preds)):
        if majority_vote:
            preds.append(np.sign(kmeans_preds[idx] + aec_preds[idx] + aew_preds[idx]))
        else:
            if kmeans_preds[idx] == -1 or aec_preds[idx] == -1 or aew_preds[idx] == -1:
                preds.append(-1)
            else:
                preds.append(1)

    return {
        "kmeans": accuracy_score(labels, kmeans_preds),
        "aec": accuracy_score(labels, aec_preds),
        "aew": accuracy_score(labels, aew_preds),
        "ensemble": accuracy_score(labels, preds)
    }

In [69]:
def get_ensemble_result(df_test, test_data_aug, use_short_flow, 
                        kmeans_load_path, aec_input_dim, aec_load_path, 
                        aew_input_dim, aew_load_path, majority_vote=False):
    
    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels  \
    = transform(get_flows(df_test), feature_type="encoding" 
                ,data_type="test", test_data_aug=test_data_aug)
    data_encoding = test_flow_data if not use_short_flow else test_flow_data + test_packet_data
    labels_encoding = test_flow_labels if not use_short_flow else test_flow_labels + test_packet_labels

    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels \
    = transform(get_flows(df_test), data_type="test", test_data_aug=test_data_aug)
    data_whisper = test_flow_data if not use_short_flow else test_flow_data + test_packet_data
    labels_whisper = test_flow_labels if not use_short_flow else test_flow_labels + test_packet_labels

    assert len(labels_encoding) == len(labels_whisper), \
        print(f"len labels_encoding: {len(labels_encoding)}, len labels_whisper: {len(labels_whisper)}")
    for idx in range(len(labels_encoding)):
        assert labels_encoding[idx] == labels_whisper[idx]
    
    acc = test_ensemble(data_encoding, data_whisper, labels_whisper, 
                        kmeans_load_path, aec_input_dim, aec_load_path, aew_input_dim, 
                        aew_load_path, test_data_aug=test_data_aug, majority_vote=majority_vote)
    return acc

In [72]:
USE_DATA_AUG = True
USE_SHORT_FLOW = True
MAJORIY_VOTE = False
accuracy_dict = {}

vote_type = "majority" if MAJORIY_VOTE else "positive"
suffix = "-all" if USE_SHORT_FLOW else "-long"
train_benign_filename = "dataset/benign_small.csv"

aec_input_dim = MAX_LEN
aew_input_dim = whisper_config["n_fft"] // 2 + 1
kmeans_save_path = os.path.join("model", "whisper", "kmeans"+suffix, 
                    os.path.basename(train_benign_filename), "kmeans.json")
aec_save_dir = os.path.join("model", "autoencoding"+suffix, 
                        os.path.basename(train_benign_filename))
aew_save_dir = os.path.join("model", "whisper", "autoencoder"+suffix, 
                        os.path.basename(train_benign_filename))

## Train Frequency features + KMeans

In [61]:
train_df = pd.read_csv(train_benign_filename)
train_df["label"] = 1
train_packet_data, train_packet_labels, train_flow_data, train_flow_labels \
= transform(get_flows(train_df))

train_data = train_flow_data if not USE_SHORT_FLOW else train_flow_data + train_packet_data
train_labels = train_flow_labels if not USE_SHORT_FLOW else train_flow_labels + train_packet_labels

In [73]:
train_kmeans(train_data, kmeans_save_path, whisper_config["val_K"])

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Train Frequency features + AutoEncoders

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_aew = AutoEncoder(aew_input_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_aew.parameters(), lr=0.01, weight_decay=1e-5)
train_ae(torch.tensor(train_data), torch.tensor(train_labels), aew_save_dir,
         model_aew, criterion, optimizer, device, num_epochs=50)

Epoch 1/50, Loss: 98.1166
Epoch 2/50, Loss: 23.9330
Epoch 3/50, Loss: 5.0799
Epoch 4/50, Loss: 1.9488
Epoch 5/50, Loss: 0.7033
Epoch 6/50, Loss: 2.9570
Epoch 7/50, Loss: 0.3255
Epoch 8/50, Loss: 2.7003
Epoch 9/50, Loss: 0.8728
Epoch 10/50, Loss: 0.5623
Epoch 11/50, Loss: 0.9623
Epoch 12/50, Loss: 0.4802
Epoch 13/50, Loss: 0.3182
Epoch 14/50, Loss: 0.6932
Epoch 15/50, Loss: 0.7875
Epoch 16/50, Loss: 0.2908
Epoch 17/50, Loss: 0.7281
Epoch 18/50, Loss: 0.2246
Epoch 19/50, Loss: 0.5675
Epoch 20/50, Loss: 0.5423
Epoch 21/50, Loss: 0.2764
Epoch 22/50, Loss: 1.2582
Epoch 23/50, Loss: 0.6044
Epoch 24/50, Loss: 0.3076
Epoch 25/50, Loss: 0.3734
Epoch 26/50, Loss: 0.3808
Epoch 27/50, Loss: 0.6041
Epoch 28/50, Loss: 0.3547
Epoch 29/50, Loss: 0.6665
Epoch 30/50, Loss: 0.6599
Epoch 31/50, Loss: 0.6247
Epoch 32/50, Loss: 0.2840
Epoch 33/50, Loss: 1.9269
Epoch 34/50, Loss: 2.5784
Epoch 35/50, Loss: 1.2231
Epoch 36/50, Loss: 0.4141
Epoch 37/50, Loss: 1.5400
Epoch 38/50, Loss: 1.2441
Epoch 39/50, Loss: 

## Train Time features + AutoEncoder

In [64]:
train_df = pd.read_csv(train_benign_filename)
train_df["label"] = 1
train_packet_data_, train_packet_labels_, train_flow_data_, train_flow_labels_ \
= transform(get_flows(train_df), feature_type="encoding")

train_data_ = train_flow_data_ if not USE_SHORT_FLOW else train_flow_data_ + train_packet_data_
train_labels_ = train_flow_labels_ if not USE_SHORT_FLOW else train_flow_labels_ + train_packet_labels_

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_aec = AutoEncoder(aec_input_dim, decoder_sigmoid=True).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_aec.parameters(), lr=0.01, weight_decay=1e-5)
train_ae(torch.tensor(train_data_), torch.tensor(train_labels_), aec_save_dir,
         model_aec, criterion, optimizer, device, num_epochs=50, decoder_sigmoid=True)

Epoch 1/50, Loss: 0.0816
Epoch 2/50, Loss: 0.0461
Epoch 3/50, Loss: 0.0254
Epoch 4/50, Loss: 0.0206
Epoch 5/50, Loss: 0.0150
Epoch 6/50, Loss: 0.0207
Epoch 7/50, Loss: 0.0185
Epoch 8/50, Loss: 0.0141
Epoch 9/50, Loss: 0.0125
Epoch 10/50, Loss: 0.0024
Epoch 11/50, Loss: 0.0128
Epoch 12/50, Loss: 0.0070
Epoch 13/50, Loss: 0.0020
Epoch 14/50, Loss: 0.0052
Epoch 15/50, Loss: 0.0029
Epoch 16/50, Loss: 0.0031
Epoch 17/50, Loss: 0.0041
Epoch 18/50, Loss: 0.0047
Epoch 19/50, Loss: 0.0039
Epoch 20/50, Loss: 0.0037
Epoch 21/50, Loss: 0.0063
Epoch 22/50, Loss: 0.0057
Epoch 23/50, Loss: 0.0041
Epoch 24/50, Loss: 0.0017
Epoch 25/50, Loss: 0.0058
Epoch 26/50, Loss: 0.0083
Epoch 27/50, Loss: 0.0056
Epoch 28/50, Loss: 0.0073
Epoch 29/50, Loss: 0.0030
Epoch 30/50, Loss: 0.0017
Epoch 31/50, Loss: 0.0021
Epoch 32/50, Loss: 0.0029
Epoch 33/50, Loss: 0.0051
Epoch 34/50, Loss: 0.0014
Epoch 35/50, Loss: 0.0028
Epoch 36/50, Loss: 0.0044
Epoch 37/50, Loss: 0.0027
Epoch 38/50, Loss: 0.0003
Epoch 39/50, Loss: 0.

# Test ensemble

In [78]:
for test_benign_filename in benign_filenames:
    test_df = pd.read_csv(test_benign_filename)
    test_df["label"] = 1
    acc = get_ensemble_result(test_df, USE_DATA_AUG, USE_SHORT_FLOW, 
            kmeans_save_path, aec_input_dim, aec_save_dir, aew_input_dim, 
            aew_save_dir, majority_vote=MAJORIY_VOTE)
    print(f"accuracy of {test_benign_filename}: {acc}")
    accuracy_dict[test_benign_filename] = acc

for test_attack_filename in attack_filenames:
    test_df = pd.read_csv(test_attack_filename)
    test_df["label"] = -1
    acc = get_ensemble_result(test_df, USE_DATA_AUG, USE_SHORT_FLOW, 
            kmeans_save_path, aec_input_dim, aec_save_dir, aew_input_dim, 
            aew_save_dir, majority_vote=MAJORIY_VOTE)
    print(f"accuracy of {test_attack_filename}: {acc}")
    accuracy_dict[test_attack_filename] = acc 

accuracy_base_name = "flow-accuracy.json" if not USE_SHORT_FLOW else "all-accuracy.json"
accuracy_save_path = os.path.join("result", "ensemble", vote_type,
                    os.path.basename(train_benign_filename), accuracy_base_name)
os.makedirs(os.path.dirname(accuracy_save_path), exist_ok=True)
with open(accuracy_save_path, "w") as f:
    json.dump(accuracy_dict, f, indent=4)

accuracy of train_set/benign1.csv: {'kmeans': 0.997979797979798, 'aec': 1.0, 'aew': 0.997979797979798, 'ensemble': 0.997979797979798}
accuracy of train_set/benign2.csv: {'kmeans': 0.9920792079207921, 'aec': 1.0, 'aew': 0.9920792079207921, 'ensemble': 0.9900990099009901}


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret

accuracy of attack_set/LDoS_small.csv: {'kmeans': nan, 'aec': nan, 'aew': nan, 'ensemble': nan}


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret

accuracy of attack_set/osscan.csv: {'kmeans': nan, 'aec': nan, 'aew': nan, 'ensemble': nan}
accuracy of attack_set/infiltration.csv: {'kmeans': 0.8333333333333334, 'aec': 0.0, 'aew': 0.8333333333333334, 'ensemble': 0.8333333333333334}


...

accuracy of attack_set/HOIC_small.csv: {'kmeans': nan, 'aec': nan, 'aew': nan, 'ensemble': nan}
accuracy of attack_set/BruteForce-Web.csv: {'kmeans': 0.5147058823529411, 'aec': 0.0, 'aew': 0.9485294117647058, 'ensemble': 0.9485294117647058}
accuracy of attack_set/LOIC_UDP_small.csv: {'kmeans': 0.7727272727272727, 'aec': 0.0, 'aew': 0.7727272727272727, 'ensemble': 0.7727272727272727}
accuracy of attack_set/SQL_Injection.csv: {'kmeans': nan, 'aec': nan, 'aew': nan, 'ensemble': nan}


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret

accuracy of attack_set/ssldosA.csv: {'kmeans': 0.0, 'aec': 0.0, 'aew': 0.0, 'ensemble': 0.0}


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/root/anaconda3/envs/PRO/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret

accuracy of attack_set/fuzzscan.csv: {'kmeans': nan, 'aec': nan, 'aew': nan, 'ensemble': nan}
accuracy of attack_set/BruteForce-XSS.csv: {'kmeans': 1.0, 'aec': 0.0, 'aew': 1.0, 'ensemble': 1.0}
